In [23]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time as time

In [24]:
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
player_data = []

for letter in letters:
    url = f"https://www.footballdb.com/players/current.html?letter={letter}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        player_rows = soup.find_all('div', class_='tr')
        for row in player_rows:
            player_name_tag = row.find('div', class_='td').find('a', href=True)
            if player_name_tag:
                player_name = player_name_tag.get_text(strip=True)
            position_tag = row.find_all('div', class_='td')[1]
            player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
            # team_tag = row.find_all('div', class_='td')[2]
            # player_team = team_tag.get_text(strip=True) if team_tag else 'N/A'
            # secondary_team_tag = row.find_all('div', class_='td')[3]
            # secondary_team = secondary_team_tag.get_text(strip=True) if secondary_team_tag else 'N/A'
            player_data.append([player_name, player_position])

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
df_names = pd.DataFrame(player_data, columns=["Player Name", "Position"])
# df.to_csv("players_data.csv", index=False, encoding="utf-8")
# print(f"Saved {len(df)} player records to players_data.csv")
print(df_names)

    

             Player Name Position
0     Abanikanda, Israel       RB
1        Abdullah, Ameer       RB
2        Abdullah, Yasir       LB
3       Abernathy, Micah       DB
4         Abraham, Micah       DB
...                  ...      ...
2543         Zentner, Ty        P
2544      Zierer, Kilian       OT
2545         Zinter, Zak       OG
2546      Zuerlein, Greg        K
2547      Zylstra, Shane       TE

[2548 rows x 2 columns]


In [27]:
# Cell takes about ten minutes to run
nfl_teams = [
    "arizona-cardinals", "atlanta-falcons", "baltimore-ravens", "buffalo-bills",
    "carolina-panthers", "chicago-bears", "cincinnati-bengals", "cleveland-browns",
    "dallas-cowboys", "denver-broncos", "detroit-lions", "green-bay-packers",
    "houston-texans", "indianapolis-colts", "jacksonville-jaguars", "kansas-city-chiefs",
    "los-angeles-chargers", "los-angeles-rams", "miami-dolphins", "minnesota-vikings",
    "new-england-patriots", "new-orleans-saints", "new-york-giants", "new-york-jets",
    "philadelphia-eagles", "pittsburgh-steelers", "san-francisco-49ers", "seattle-seahawks",
    "tampa-bay-buccaneers", "tennessee-titans", "washington-commanders"
]

counter = 0
player_data = []
D = 2003
for yearo in range(D, 2024):
    year = 4026 - yearo
    time.sleep(10) # Add some sleep time to not overload request limit
    for team in nfl_teams: 
        url = f"https://www.footballdb.com/teams/nfl/{team}/roster/{year}"
        if team == "washington-commanders" and year >= 1936 and year < 2020:
            url = f"https://www.footballdb.com/teams/nfl/washington-redskins/roster/{year}"
        elif team == "washington-commanders" and (year == 2021 or year == 2020):
            url = f"https://www.footballdb.com/teams/nfl/washington-football-team/roster/{year}"
        elif team == "los-angeles-rams" and year >= 1995 and year <= 2015:
            url = f"https://www.footballdb.com/teams/nfl/st-louis-rams/roster/{year}"
        elif team == "los-angeles-chargers" and year <= 2016:
            url = f"https://www.footballdb.com/teams/nfl/san-diego-chargers/roster/{year}"
        elif team == "tennessee-titans" and year <= 1998:
            url = f"https://www.footballdb.com/teams/nfl/tennessee-oilers/roster/{year}"    
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            print(f"Got data for the {team} in year {year}")
            soup = BeautifulSoup(response.text, 'html.parser')
            roster_table = soup.find('div', class_='divtable divtable-striped divtable-mobile')
            player_rows = roster_table.find_all('div', class_='tr')
            
            for row in player_rows:
                # number_tag = row.find('div', class_='td w10 m20 rostercell_num')
                # player_number = number_tag.get_text(strip=True) if number_tag else 'N/A'
                name_tag = row.find('div', class_='td w20 m80 rostercell_name')
                player_name_tag = name_tag.find('a', href=True) if name_tag else None
                player_name = player_name_tag.get_text(strip=True) if player_name_tag else 'N/A'
                # position_tag = row.find('div', class_='td w10 rostercell_pos hidden-xs')
                # player_position = position_tag.get_text(strip=True) if position_tag else 'N/A'
                # height_tag = row.find('div', class_='td w10 rostercell_ht hidden-xs')
                # player_height = height_tag.get_text(strip=True) if height_tag else 'N/A'
                # weight_tag = row.find('div', class_='td w10 rostercell_wt hidden-xs')
                # player_weight = weight_tag.get_text(strip=True) if weight_tag else 'N/A'
                # age_tag = row.find('div', class_='td w10 rostercell_age hidden-xs')
                # player_age = age_tag.get_text(strip=True) if age_tag else 'N/A'
                # college_tag = row.find('div', class_='td w30 rostercell_coll hidden-xs')
                # player_college = college_tag.get_text(strip=True) if college_tag else 'N/A'
                player_data.append([team, year, player_name])

        else:
            print(f"FAILED to retrieve the page for team {team} in year {year}. Status code: {response.status_code}")
df = pd.DataFrame(player_data, columns=["Team", "Year", "Player Name"])
print(df.shape)

Got data for the arizona-cardinals in year 2023
Got data for the atlanta-falcons in year 2023
Got data for the baltimore-ravens in year 2023
Got data for the buffalo-bills in year 2023
Got data for the carolina-panthers in year 2023
Got data for the chicago-bears in year 2023
Got data for the cincinnati-bengals in year 2023
Got data for the cleveland-browns in year 2023
Got data for the dallas-cowboys in year 2023
Got data for the denver-broncos in year 2023
Got data for the detroit-lions in year 2023
Got data for the green-bay-packers in year 2023
Got data for the houston-texans in year 2023
Got data for the indianapolis-colts in year 2023
Got data for the jacksonville-jaguars in year 2023
Got data for the kansas-city-chiefs in year 2023
Got data for the los-angeles-chargers in year 2023
Got data for the los-angeles-rams in year 2023
Got data for the miami-dolphins in year 2023
Got data for the minnesota-vikings in year 2023
Got data for the new-england-patriots in year 2023
Got data 

In [ ]:
print(df)
df.to_csv(f"team_composition_{D}_to_2023.csv", index=False, encoding="utf-8")

Empty DataFrame
Columns: [Team, Year, Player Name]
Index: []
